# Indexing Scholarly Research Articles

Pipeline for creating and analysing scientific networks. The first phase is downloading the publicly available data. The second phase is to extract the data of the provided date range. The third phase is to filter the cited and citing nodes of DOIs. The fourth phase is to load the data as scientific networks. Finally, the last phase is to index the articles.
Publicly available dumps of both Crossref and COCI are free for use. Other than downloading the data, it is also needed to have seed DOIs. The steps are not dependent on each other but subsequent phases depend on data being placed in correct directories.

In [2]:
%load_ext autoreload
%autoreload 2

import Extract, Transform, Load, Index

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Input

In [3]:
#Input use case name to append 
usecase="BIB2" 

'''
BIB1: Bibliometric Study with seminal seed DOIs
BIB2: Bibliometric Study with random seed DOIs
IND1: Indicators Review with seminal seed DOIs
IND2: Indicators Review with random seed DOIs
COM: Community Review with random seed DOIs
'''

#Input random network Flag
randomFlag = False

#Input citation cascade details / total ego levels 
totalLevels = 3

# Input DASK configurations
physicalCores = 8
virtualPerCore = 2

#Input data directory to use
DataDir = "../DATA/"
usecaseDIR= "../"+usecase+"/"
path_to_crossref = DataDir+"Crossref/crossref_public_data_file_2021_01/*.json.gz"
path_to_COCI = DataDir+"COCI/csv/*.csv"

print("Please have following files created")
print(DataDir)
print(usecaseDIR)
print(path_to_crossref)
print(path_to_COCI)

Please have following files created
../DATA/
../BIB2/
../DATA/Crossref/crossref_public_data_file_2021_01/*.json.gz
../DATA/COCI/csv/*.csv


# Extraction Phase

This phase concerns extracting the metadata for an input date range. Only the past decade is considered to give equal chance to early career researchers. It is also needed to curtail processing all the data. The date range should be selected based on the available processing and RAM. To process large files, DASK is used for out-of-core processing. It can be scaled to multi-node clusters. The steps are not dependent on each other but subsequent phases depend on relevant data being extracted that can fit in memory.


In [4]:
#Date range
if usecase[0:3]=="BIB":
    start_year = 2010
    end_year = 2020
elif usecase[0:3]=="IND":
    start_year = 1985
    end_year = 2015
elif usecase[0:3]=="COM":
    start_year = 1970
    end_year = 2009
else:
    start_year = 1985
    end_year = 2020


In [5]:
#Directory sub folders path
path_to_crossref_parquet = DataDir+"Crossref/"+str(start_year)+"parquet"+str(end_year)+"/"
metadataFull = DataDir+"Crossref/"+str(start_year)+"metadata"+str(end_year)+".pkl"
citNetFull = DataDir+"COCI/"+str(start_year)+"COCI"+str(end_year)+".pkl"

CrossrefColsDF = ['DOI','Title','Venue','Authors','Year']

path_to_COCI_parquet = DataDir+"COCI/"+str(start_year)+"parquet"+str(end_year)+"/"
COCIColsDDF=['citing','cited','creation', 'oci']

#### Extract metadata for the provided date range in parquet format
This step relates to creating a parquet binary file format for Crossref JSON. It is needed for the fast processing of subsequent steps. For a date range 2010-2020 this requires around 15 GB of space. This is a compute-heavy step and takes approximately five hours to complete. It uses a DASK Bag to read the JSON files.

In [25]:
%time Extract.convertCrossrefJSONDumpToParquetDDF(path_to_crossref, path_to_crossref_parquet, physicalCores, virtualPerCore, start_year, end_year)

C:\ProgramData\Anaconda3\lib\site-packages\distributed\node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56608 instead
  f"Port {expected} is already in use.\n"


<Client: 'tcp://127.0.0.1:56611' processes=8 threads=16, memory=67.95 GiB>


ValueError: ('No files found', '../DATA/Crossref/crossref_public_data_file_2021_01/*.json.gz')

#### Convert parquet to pickle format
This step relates to extracting relevant columns of metadata to pickle format. It is a memory-intensive step and takes approximately an hour to complete. It creates a 16 GB binary file that is used by subsequent phases.


In [26]:
%time Extract.convertParquetToPickleDF (path_to_crossref_parquet, CrossrefColsDF, metadataFull)

OSError: [Errno 22] Invalid argument: '../DATA/Crossref/2010parquet2020/'

#### Extract COCI for the provided date range in parquet format
This step relates to creating a parquet binary file format for COCI. It is needed for the fast processing of subsequent steps. For a date range from the year 2010 to the year 2020, this requires around 45 GB of space. This is a compute-heavy step and takes approximately three hours to complete. It uses a DASK Dataframe to read the CSV files.

In [27]:
%time Extract.convertCOCIDumpToParquetDDF (path_to_COCI, COCIColsDDF, physicalCores, virtualPerCore, path_to_COCI_parquet, start_year, end_year)

C:\ProgramData\Anaconda3\lib\site-packages\distributed\node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56700 instead
  f"Port {expected} is already in use.\n"


<Client: 'tcp://127.0.0.1:56703' processes=8 threads=16>


IndexError: list index out of range

# Transformation Phase

This phase concerns transforming the data acquired to a format that is suitable for loading to the network processing library. The steps are dependent on their preceding steps. This phases takes around 2 hours to complete.

In [6]:
DOIPkl = usecaseDIR+"DOIs.pkl"
DOItestPkl = usecaseDIR+"DOItest.pkl"
refDf = usecaseDIR+"refDf.pkl"

In [7]:
#reference DOI or URL for comparisons 
if usecase[0:3]=="BIB2":
    refDOI = '10.1016/j.physrep.2009.11.002' #for COM
else:
    refDOI = '10.1016/j.joi.2016.02.007' #for IND

refURL = "https://ndownloader.figstatic.com/files/30873946" #for BIB
DOIColumn = "DI"

DOIList = ['10.1126/science.122.3159.108','10.1073/pnas.0507655102']

if usecase[3:4]=="1": #use Seminal DOIs
    sample_n=0
elif usecase[0:3]=="BIB":
    sample_n=5
else:
    sample_n=2
    
sample_random_state = 200 #used as random split by pandas

In [8]:
#make df
citNetFull = usecaseDIR+str(start_year)+"COCIFull"+str(end_year)+".pkl"
metaData = usecaseDIR+str(start_year)+"metadata"+str(end_year)+".pkl"
citNet = usecaseDIR+str(start_year)+"COCI"+str(end_year)+".pkl"

#### Convert DOI List to pickle
This step relates to converting the seed DOIs to pickle format. This step is needed before transforming the data.

In [31]:
if usecase[0:3]=="BIB2":
    %time Transform.getDOIsFromCSV (refURL, refDf, DOIColumn)
else:
    %time Transform.getDOIsFromRef (path_to_COCI, COCIColsDDF, physicalCores, virtualPerCore, refDOI, refDf)

C:\ProgramData\Anaconda3\lib\site-packages\distributed\node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56813 instead
  f"Port {expected} is already in use.\n"


<Client: 'tcp://127.0.0.1:56816' processes=8 threads=16>


IndexError: list index out of range

In [32]:
%time Transform.splitDOIs (refDf, DOIPkl, DOItestPkl, metadataFull, sample_n, sample_random_state)

FileNotFoundError: [Errno 2] No such file or directory: '../DATA/Crossref/2010metadata2020.pkl'

In [33]:
if usecase[3:4]=="1": #use Seminal DOIs
    %time Transform.getDOIsFromList (DOIList, DOIPkl)

#### Filter COCI using DOIs ego network
This step relates to iteratively adding the references and citations of DOIs and updating the DOI List. 

In [34]:
%time Transform.filterCOCIDump (path_to_COCI_parquet, physicalCores, virtualPerCore,citNetFull, DOIPkl, totalLevels)

KeyboardInterrupt: 

#### Filter metadata using COCI DOIs
This step relates to taking metadata of relevant DOIs. It is needed to create the corresponding venue citation network and author citation network for the publication citation network created by the previous step.

In [35]:
%time Transform.filterMetadata (metadataFull, citNetFull, metaData, citNet)

KeyboardInterrupt: 

In [ ]:
%time Transform.filterZeroOutDegNodesFromCOCI (metaData, citNet)

# Loading Phase

This phase concerns the loading of data into a network processing library using the following Equations. Scripts have been optimised to work with millions of edges with 64 GB memory. The unique list of nodes is first hashed to an increasing number representing the node labels. Edges are created between the integer labels. This phases takes a few minutes to complete.


In [1]:
# Initialization for Loading
autCitNet = usecaseDIR+"autCOCI.pkl"
autCitNetLst = usecaseDIR+"autCOCILst.pkl"

if usecase[3:4]=='2': #reduced cutoff for random DOIs
    refCutoffPub = 1
    refCutoffVen = 1
    refCutoffAut = 1
else:
    refCutoffPub = 3
    refCutoffVen = 5
    refCutoffAut = 7

pubF = usecaseDIR+"Publication"
venF = usecaseDIR+"Venue"
autF = usecaseDIR+"Author"

NameError: name 'usecaseDIR' is not defined

#### Generate publication citation network
This step relates to generating the publication citation network using the COCI data. The COCI data has a few self-loops and bi-directional loops which are removed.

In [ ]:
%time Load.generatePublicationCitationNetwork(pubF, metaData, citNet, randomFlag, refCutoffPub)

#### Generate venue citation network
This step relates to generating the venue citation network using the corresponding journal of the filtered publication citation network. Even though self-citations at the level of an individual or publisher are sometimes critiqued, it exists as a valid form of recognition and are kept.

In [ ]:
%time Load.generateVenueCitationNetwork(venF, citNet, refCutoffVen)


#### Author disambiguation
One of the key challenges in the bibliometric analysis is the author name disambiguation. Crossref does not provide any disambiguation like Microsoft Academic Graph \cite{wang_microsoft_2020}. If ORCID is available in the Crossref it is recorded along with the given name and last name of the authors. If the name has multiple ORCID associated then the ORCID is used as an identifier instead of a name. This way authors within the field are disambiguated using a simple approach, similar to \cite{milojevic_accuracy_2013}.

In [ ]:
%time Load.generateAuthorID (metaData, citNet, autCitNet, autCitNetLst)


#### Generate author citation network
This step relates to generating the author citation network using the corresponding authors of the filtered publication citation network. Cartesian product between the authors of citing and cited publication is made. In case of publication having a large number of authors, the list is kept up to 10 authors including the last two authors. This step is necessary to keep the network in memory.

In [ ]:
%time Load.generateAuthorCitationNetwork(autF, autCitNet, autCitNetLst, refCutoffAut)

# Index

This phase concerns with creating an index using the following Equations. Steps are to be executed sequentially. Percentile ranks are used instead of the raw score. This phases takes a few minutes to complete.


In [9]:
# following files are needed in the CWD

VenueGraph = usecaseDIR+"Venue.graph"
VenueHash = usecaseDIR+"Venue.hash"

PublicationGraph = usecaseDIR+"Publication.graph"
PublicationHash = usecaseDIR+"Publication.hash"

AuthorGraph = usecaseDIR+"Author.graph"
AuthorHash = usecaseDIR+"Author.hash"

# following files will be created in the CWD
AuthorInfoCSV = usecaseDIR+"Author.info"
VenueInfoCSV = usecaseDIR+"Venue.info"
PublicationInfoCSV = usecaseDIR+"Publication.info"
PublicationRankCSV = usecaseDIR+"Publication.csv"
corrF = usecaseDIR+"Corr.svg"
gridF = usecaseDIR+"Grid.svg"

alpha = 0.85
beta = 0.5
gamma = 0.5



#### Calculate author and venue score using PageRank
This step relates to indexing the venue using the PageRank score. A node with citations from a high score node also gets a high score. This recursive mechanism guarantees that not only highly cited nodes are indexed but also nodes cited by other important nodes are also indexed. 

In [10]:
%time Index.generateVenueRank(VenueHash, VenueGraph, VenueInfoCSV, alpha)

Venue Ranking Completed
Wall time: 12 ms


In [11]:
%time Index.generateAuthorRank(AuthorHash, AuthorGraph, AuthorInfoCSV, alpha)

Author Ranking Completed
Wall time: 52 ms



#### Calculate publication score using author and venue score
This step relates to indexing publications based on the two scores. First, the score of the venue where it is published and the cumulative score of which authors published it. Second, the cumulative score of venues and authors by which the publication is cited. 

In [12]:
%time Index.generatePublicationRank(VenueInfoCSV, metaData, citNet, PublicationGraph, PublicationHash, AuthorInfoCSV, PublicationInfoCSV, beta, gamma, True)

Publication Ranking Completed
Wall time: 463 ms


# Results

In [13]:
%time Index.generateQualitativeResults(PublicationInfoCSV, PublicationRankCSV, metaData, corrF, gridF)

ValueError: zero-size array to reduction operation minimum which has no identity

In [14]:
%time Index.generateComparison(metaData, PublicationInfoCSV, DOItestPkl, True)

0 10013 0 0 1693 0.0
Wall time: 184 ms


0

In [15]:
%time Index.generateQuantitativeResults (VenueInfoCSV, metaData, citNet, PublicationGraph, PublicationHash, AuthorInfoCSV, PublicationInfoCSV, DOItestPkl)

min
 beta     0.0
gamma    0.0
val      0.0
dtype: float64
max
 beta     1.0
gamma    1.0
val      0.0
dtype: float64
mean
 beta     0.5
gamma    0.5
val      0.0
dtype: float64
     beta  gamma  val
0     0.0    0.0  0.0
1     0.0    0.1  0.0
2     0.0    0.2  0.0
3     0.0    0.3  0.0
4     0.0    0.4  0.0
..    ...    ...  ...
116   1.0    0.6  0.0
117   1.0    0.7  0.0
118   1.0    0.8  0.0
119   1.0    0.9  0.0
120   1.0    1.0  0.0

[121 rows x 3 columns]
Wall time: 53.8 s


In [16]:
%time Index.generateBaseline (citNetFull, DOIPkl, DOItestPkl)

refs in Bibliographic Coupling Top 1000 108
refs in Cocited list Top 390 65
Wall time: 8.08 s
